In [33]:
from transformers import *
import os
import copy
import numpy as np
import pandas as pd
import tokenizers

In [2]:
model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )


def get_prediction_v3(tokenizer, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
        orig_word = tokenizer.convert_tokens_to_string(sub_words)
#         print(sub_words, " " + word)
#         print(" " + word, orig_word)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        cursor += len(" " + word)
        end_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            tweet_offsets.append((start_offsets, end_offsets))
            
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0 and orig_word != " " + word:
                sub_word_len += 1
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            token_level_cursor += sub_word_len
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
        
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    print(tweet_offsets[targets_start][0], tweet_offsets_token_level[targets_start][0])
    print(tweet_offsets[targets_end][1], tweet_offsets_token_level[targets_end][1])
    
    
    return tweet[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]], \
            tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "Happy Mother`s Day to all the Momma`s to be and Mommas currently.  Have a wonderful day!!!"
selected_text = "wonderful"

get_prediction_v3(tokenizer, tweet, selected_text)

73 73
83 83


(' wonderful', ' wonderful')

In [3]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = " miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers."


input_ids_orig_0 = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(tweet))

ids_0 = []
offsets_0 = []
offsets_0_token_level = []
cursor = 0


for i, word in enumerate(tweet.lower().split()):
    
    encoded_word = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(" " + word))
    ids_0 += encoded_word
    
    number_of_tokens = len(encoded_word)

    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor
    
    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        offsets_0.append((start_offsets, end_offsets))
        curr_token = tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(encoded_word[i]))
        
        offsets_0_token_level.append((token_level_cursor, token_level_cursor + len(curr_token)))
        token_level_cursor += len(curr_token)
    
# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(ids_0)
print(offsets_0_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(ids_0)))

ids_1 = tokenizer_1.encode(tweet).ids
print(ids_1)
print(tokenizer_1.encode(tweet).offsets)
print(tokenizer_1.decode(ids_1))

[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (101, 104), (104, 110), (110, 115), (115, 121), (121, 122)]
 miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers.
[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (10

In [4]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [5]:
# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

# for i in range(len(train)):
    
#     prediction, prediction_token = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i])
#     prediction = prediction.strip()
#     prediction_token = prediction_token.strip()
#     print(prediction, "--------------------", prediction_token)
    
#     label = str(train["selected_text"].iloc[i]).lower().strip()
#     if label != prediction:
#         print(
#               label, \
#               "-------------------", \
#               get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
#               "-------------------", train["text"].iloc[i])

In [6]:
model_type = "roberta-base"

tokenizer_1 =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)


def check_offsets(tokenizer_1, tokenizer_2, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for i, word in enumerate(tweet.split()):

        sub_words = tokenizer_1.tokenize(" " + word)
#         print(sub_words)
        encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word

        start_offsets = cursor
        end_offsets = cursor

        token_level_cursor = start_offsets

        for i in range(number_of_tokens):

            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len

            
    tweet_offsets = tokenizer_2.encode(tweet).offsets
    
    if str(tweet_offsets_token_level) != str(tweet_offsets):
        print(tweet)
        print(tweet_offsets_token_level)
        print(tweet_offsets)
        
    
#     get word idx
#     target_idx = []
#     for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
# #         print(tweet[offset1: offset2])
#         if sum(char_targets[offset1: offset2]) > 0:
#             target_idx.append(j)
    
#     targets_start = target_idx[0]
#     targets_end = target_idx[-1]
    
# tweet = "iï¿½m trying to"
# selected_text = "iï¿½m trying to"
# check_offsets(tokenizer_1, tokenizer_2, tweet, selected_text)
# for i in range(len(train)):

#     check_offsets(tokenizer_1, tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i])

In [37]:
model_type = "roberta-base"

tokenizer_1 =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)


def check_label(tokenizer_1, tokenizer_2, tweet, selected_text, num):
    
    # lower cased here
    tweet = " ".join(str(tweet).split()).lower()
    selected_text = " ".join(str(selected_text).split()).lower()
    original_tweet = copy.deepcopy(tweet)
    original_selected_text = copy.deepcopy(selected_text)

    # some token optimization
    special_tokens = ["!!!!", "!!!", "!!", "!", \
                      "....", "...", "..", ".", \
                      "????", "???", "??", "?", \
                      "****", "***", "**", "*", \
                      "----", "---", "--", "-", \
                      "~~~~", "~~~", "~~", "~", \
                      "////", "///", "//", "/", \
                      "````", "```", "``", "`", \
                      "====", "===", "==", "=", \
                      "####", "###", "##", "#", \
                      "&&&&", "&&&", "&&", "&", \
                      "))))", ")))", "))", ")", \
                      ";;;;", ";;;", ";;", ";", \
                      "''''", "'''", "''", "'", \
                      "<<<<", "<<<", "<<", "<"
                     ]
    for token in special_tokens:
        replace_token = " ".join(token) + " "
        tweet = tweet.replace(token, replace_token)
        selected_text = selected_text.replace(token, replace_token)

    tweet = " " + " ".join(tweet.split())
    selected_text = " " + " ".join(selected_text.split())
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for i, word in enumerate(tweet.split()):

        sub_words = tokenizer_1.tokenize(" " + word)
        encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word

        start_offsets = cursor
        end_offsets = cursor

        token_level_cursor = start_offsets

        for i in range(number_of_tokens):

            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len

    tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     pred = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]
    
#     pred = str(pred).lower().strip()
#     selected_text = str(selected_text).lower().strip()
    
#     if pred != selected_text:
#         print(pred)
#         print(selected_text)
#         num += 1

    prediction = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]].strip()
    filtered_output = ""
    finished = False

    # print("tweet: ", tweet)
    # print("prediction: ", prediction)

    if len(prediction) == 0:
        filtered_output = prediction
    else:
        max_match = 0
        for idx in range(len(original_tweet)):
            curr_output = ""
            if original_tweet[idx] == prediction[0]:
                tweet_idx = idx
                for prediction_idx in range(len(prediction)):
                    if (original_tweet[tweet_idx] == prediction[prediction_idx]):
                        curr_output += original_tweet[tweet_idx]
                        tweet_idx += 1

                        # reach tweet end
                        if (tweet_idx >= len(original_tweet)):
                            if (prediction_idx == len(prediction) - 1):
                                finished = True
                            break

                        # reach prediction end
                        if (prediction_idx == len(prediction) - 1):
                            finished = True
                            break
                    else:
                        # skip extra " "
                        if (prediction[prediction_idx] == " "):
                            continue
                        else:
                            break
                if finished:
                    filtered_output = curr_output
                    break
                else:
                    if max_match < len(curr_output):
                        max_match = len(curr_output)
                        filtered_output = curr_output
                        
    if filtered_output.strip() != original_selected_text.strip():
        print(filtered_output)
        print(original_selected_text)
        num += 1
    
    return num

num = 0
for i in range(len(train)):

    num = check_label(tokenizer_1, tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i], num)
    
print(num)

gonna
onna
well so much for being unhappy for about 10 minutes
well so much for being unhappy for about 10 minute
a little happy for
a little happy fo
dont like going
dont like go
and i`m not thrilled at all with mine.
d i`m not thrilled at all with mine.
totally adore
y adore
be nice
e nice
up sounds like fun
p sounds like fun
have fun
e fun
and thank you!
d thank you!
i`m not sleeping at all until
i`m not sleeping at all un
blog what is this powerblog challenge you keep talking about? i`m a newbie follower
g what is this powerblog challenge you keep talking about? i`m a newbie followe
getting harmed
g harmed
was worth
as wort
perfect match
perfect ma
please review
please re
yea, i should know.. but tell me everything! ps: send me direct messages telling.. haha
yea, i should know.. but tell me everything! ps: send me direct messages telling.. ha
have a good one
have a good on
guess i just don`t entertain him :/
s i just don`t entertain him :/
i love mine, too . happy motherï¿½s day to

i am proud of myself and
i am proud of myself an
terribly sad
ly sa
slovakian country side is a refreshing change compared
slovakian country side is a refreshing change co
we enjoyed
we enjoye
me..especially for the love of conversation
e..especially for the love of conversation
think yourself lucky! i also broke all my fingers at school when a window sash broke with my fingers like pancakes on the sill
think yourself lucky! i also broke all my fingers at school when a window sash broke with my fingers like pancakes on the sil
wow. i hope he gets better
wow. i hope he gets bette
god bless
god ble
and enjoy
d enjoy
so sorry!
o sorry!
have fun
e fun
thanks hoping
thanks hopin
having dirty
ng dirt
the best screen name
e best screen name
and meditate
d meditate
salute all
salute al
seen loads of new photos and stuff of new moon and cant wait lol (l) taylor lautner lol takes his shirt off 3 times yum
seen loads of new photos and stuff of new moon and cant wait lol (l) taylor lautner lol tak

'may the 4th' be with
'may the 4th' be wit
blessings and joy
blessings and jo
aha omg you win
a omg you win
weekend without em`
d without em`
history`s great
y`s gre
good thi
good
i feel u but its sooo close to me hard not to but i cant so no point ugh
i feel u but its sooo close to me hard not to but i cant so no point ug
love it when
love it wh
ww hope uve hada good day
w hope uve hada good day
commercial shoot in tokyo. not much time to play there quick and dirty
commercial shoot in tokyo. not much time to play there quick and dirt
she doesnt try to violently
e doesnt try to violently
then, hope it doesn`t rain
en, hope it doesn`t rain
be fine
e fine
no, seriously you guys, i /wanted/ to kick monday off with
no, seriously you guys, i /wanted/ to kick monday off wit
really hope
y hope
a wonderful
a wonderfu
soooo romantic even
soooo romantic ev
buddy. lol
y. lol
>>>> help?
>> help?
love
lov
yes, sir! see here
yes, sir! see her
bored
bore
mainly shade loving native wildflowers
ly shad

so awesome
o awesome
his love
s love
so good
o good
so lonely
o lonely
just isn`t getting rid
st isn`t getting rid
ays im chillin
ys im chillin
i can`t eat
i can`t ea
doctors don`t get sick! hope you get well soon
doctors don`t get sick! hope you get well soo
lonely,dh
lonely,d
good night twitter
good night tw
i hate
i hat
feel really unloved
l really unloved
happen hahahahah
n hahahahah
i would love to sleep with
i would love to sleep wi
a night alone with
a night alone wi
im crying
m crying
be happy
e happy
i wanna start cryin right now :/ ash
i wanna start cryin right now :/ as
hates these
hates th
ll miss
l miss
and she very graceful accepted it
d she very graceful accepted it
i am amused. but yeah, still depressed
i am amused. but yeah, still depresse
was fun
s fun
to the doctor anyone wanna do somethin after for a bit?
o the doctor anyone wanna do somethin after for a bit?
_ no idea what
_ no idea wh
time to leave a passive agressive note to the owners. it`s not the dog`s fault..

me sickee
e sickee
bullet train from tokyo the gf and i have been visiting japan since thursday vacation/sightseeing gaijin god
bullet train from tokyo the gf and i have been visiting japan since thursday vacation/sightseeing gaijin go
. oh i`m so happy
. oh i`m so happ
its nice
ts nic
i can`t do any more
i can`t do any mor
i hear you sister!! same here
i hear you sister!! same her
. hopefully
. hopefull
are always amazing
e always amazing
be great at it
e great at it
to quite like lily allen and her music, to be honest.
o quite like lily allen and her music, to be honest.
asi love my friends,i hate how they unintentionally make me feel inadequate.ineedhelp
si love my friends,i hate how they unintentionally make me feel inadequate.ineedhelp
i love
i lo
be fab
e fab
and i must laugh.you
and i must laugh.y
ruined theatre
ruined th
pictures of your bliss? in jpeg format? sorry, it`s message board terminology and u don`t play w/us on the boards
pictures of your bliss? in jpeg format? sorry

rr i hate that stuff. but i love you tho.
r i hate that stuff. but i love you tho.
not a happy bunny
not a happy bu
other fav song
er fav song
. sorry
. sorr
i hate when
i hate wh
miserable for
miserable fo
enjoyed last night at what i call 'real world social networking' - lok event in vancouver. it was great!. but headeache today
enjoyed last night at what i call 'real world social networking' - lok event in vancouver. it was great!. but headeache toda
looks fabulous
s fabulous
desperate,that
desperate,t
looks good
s good
were fab
e fab
gogol bordello < par. prolly not even in the top 10 of gb shows i`ve seen. perhaps some old fashioned civilization ii will help
gogol bordello < par. prolly not even in the top 10 of gb shows i`ve seen. perhaps some old fashioned civilization ii will hel
its good
s good
are great!
e great!
far better
r better
! have a very happy mother`s day
! have a very happy mother`s da
feelin nice after
feelin nice af
so, just pointed out to me that she has not bee

and never at a good time
d never at a good time
are terrible
e terrible
!! love you guys
!! love you guy
**** compared
**** co
have a great
ve a gre
just saw a male baltimore oriole at our suet feeder and something enjoyed a lot of thistle from our new thistle feeder early this a.m.
ust saw a male baltimore oriole at our suet feeder and something enjoyed a lot of thistle from our new thistle feeder early this a.m.
all good
l good
hate different
hate di
the best
e best
me love
e love
proudly watching
proudly wa
re idiots
re idiot
much hopeful
h hopeful
it`s freakin` hot and humid today.
t`s freakin` hot and humid today.
and this one had been a fave for a longgggg time
d this one had been a fave for a longgggg time
more worried
e worried
suck **** next
suck **** ne
shoes ugh
s ugh
yeah i love cali so much
h i love cali so much
totally excited
y excited
d it was good, but his omelette was burnt.
it was good, but his omelette was burnt.
though don`t you worry!! haha i wish i could fly to o

doomed if th
doomed
sounds like sonics...which i never had ...and for the record, i say spicy bred buffalo wings w/ blu cheese & beer
sounds like sonics...which i never had ...and for the record, i say spicy bred buffalo wings w/ blu cheese & bee
me sad
e sad
i love
i lov
ww i have no internet right now
w i have no internet right now
and miss u
d miss u
aka the best place ever
a the best place ever
you #awesomeupdater
u #awesomeupdater
so we don`t annoy our followers. thought of that too late. good night
so we don`t annoy our followers. thought of that too late. good nigh
just mean
st mea
i gave up cable in these tough economic times. it was either cable or shoes, and you know what cable lost
i gave up cable in these tough economic times. it was either cable or shoes, and you know what cable los
and thanks
d thanks
waaa the
waaa th
real sorry
l sorry
really good about
really good ab
clocked out for
clocked out fo
the best!
he best!
show was a success.
how was a success.
.. great
.. gre

. thank you
. thank yo
was hoping
s hoping
really doesn`t like shiny happy people.
y doesn`t like shiny happy people.
jamie @ sean cody, up for some angry ****?: jamie @ sean cody, i wouldn`t piss this one off hey there guys, do.. http://tinyurl.com/ddyyd6
amie @ sean cody, up for some angry ****?: jamie @ sean cody, i wouldn`t piss this one off hey there guys, do.. http://tinyurl.com/ddyyd6
honor to d
honor
haha, it`ll be gross by the time it comes back. tell you what .. we`ll yum! vancouver some malay spot soon, or just go indian
haha, it`ll be gross by the time it comes back. tell you what .. we`ll yum! vancouver some malay spot soon, or just go india
very nervous
ry nervou
31 marks the end of our roommate era. and you never made wontons for me again.
1 marks the end of our roommate era. and you never made wontons for me again.
definitely one of my favorite cities in the world.
y one of my favorite cities in the world.
devices failed
es faile
can`t wait to see the smile on her face


In [8]:
tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)

def get_target_idx(tweet, selected_text, offsets):
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1

    target_idx = []
    for j, (offset1, offset2) in enumerate(offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)

    start_idx = target_idx[0]
    end_idx = target_idx[-1]

    return start_idx, end_idx


def check_label_v2(tokenizer_2, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets

    targets_start, targets_end = get_target_idx(tweet, selected_text,tweet_offsets_token_level)
    
    pred = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]+1]
    
    pred = str(pred).lower().strip()
    selected_text = str(selected_text).lower().strip()
    
    if pred != selected_text:
        print(pred)
        print(selected_text)

for i in range(len(train)):

    check_label_v2(tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i])

bullying me.
bullying me
gonna
onna
...no internet
.no internet
well so much for being unhappy for about 10 minutes.
well so much for being unhappy for about 10 minute
a little happy for
a little happy fo
dont like going
dont like go
and i`m not thrilled at all with mine.
d i`m not thrilled at all with mine.
i lost all my friends, i`m alone and sleepy..i
i lost all my friends, i`m alone and sleepy..
jealous....
jealous..
s awesome.
s awesome
miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers.
miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers
hi how are you doing ??? *just joined twitter...*
hi how are you doing ??? *just joined twitter...
sad..
sad.
totally adore
y adore
painful...b
painful.
be nice
e nice
endearing--b
endearing-
...really bad
.really bad
up sounds like fun
p sounds like fun
bored...i
bored..
have fun
e fun
i am such a creeper i f

too shabby
o shabby
the best
the be
just finished emergency - by neil strauss - another great read neil - congrats miss you by the way!
just finished emergency - by neil strauss - another great read neil - congrats miss you by the way
bad times for everyone whywhywhy do we have to write essays no fair!
bad times for everyone whywhywhy do we have to write essays no fair
more important
e important
-- stereos = life <333
-- stereos = life <33
!! i like the beach,
! i like the beach,
so cute!
so cut
la isn`t cooperating
a isn`t cooperating
i don`t know what to do...
i don`t know what to do..
but jeff will have a hard time from 42nd.
but jeff will have a hard time from 42nd
looks like is hiding money in an offshore bank account on behalf of last.fm http://bit.ly/cet0s
looks like is hiding money in an offshore bank account on behalf of last.fm http://bit.ly/cet0
...impressive
..impressive
... you were correct
. you were correct
not good.
not good
jealous!!
jealous!
wish good
ish go
?! that l

omg..
and absolutely loved it
d absolutely loved it
mine just doesn`t want to work
ne just doesn`t want to work
me crazy!
e crazy
there hope
e hope
hurt...
hurt..
so funny...
so funny..
most epic weekend
most epic we
away. i miss you too
y. i miss you too
sounds amazing.
ds amazin
. i love providence.
. i love providence
'may the 4th' be with
'may the 4th' be wit
overreaction...
overreaction..
blessings and joy
blessings and jo
i don`t like chris.
i don`t like chris
puncture!!!
puncture!!
aha omg you win
a omg you win
saddd..i
saddd.
weekend without em`
d without em`
headache and colds...
headache and colds..
not at my workplace. but a short-sleeved shirt is absolutely necessary today!
not at my workplace. but a short-sleeved shirt is absolutely necessary today
history`s great
y`s gre
ur taking tease lessons from the guys! no fair lol, oh well, we`ll wait as patiently as we can!
ur taking tease lessons from the guys! no fair lol, oh well, we`ll wait as patiently as we can
...your blog 

im crying
m crying
be happy
e happy
it`s okay. maybe our schedules will work out soon.
it`s okay. maybe our schedules will work out soon
bin stressd...
bin stressd.
i wanna start cryin right now :/ ashl
i wanna start cryin right now :/ as
hates these
hates th
...not nice
.not nice
so frustrated!!!
so frustrated!!
ll miss
l miss
and she very graceful accepted it
d she very graceful accepted it
i am amused. but yeah, still depressed
i am amused. but yeah, still depresse
was fun
s fun
love you too!!!
love you too!
. whatthefuck.
. whatthefuck
fave!!
fave!
to the doctor anyone wanna do somethin after for a bit?
o the doctor anyone wanna do somethin after for a bit?
sit there doing.. nothing!
sit there doing.. nothing
only he keeps his clothes on but he can take them...
only he keeps his clothes on but he can take them.
m sad.
m sad
_ no idea what
_ no idea wh
time to leave a passive agressive note to the owners. it`s not the dog`s fault... it`s their **** owners
time to leave a passive agr

its always good
s always good
and was in the hospital.
nd was in the hospital
goodnite
goodnit
dont be gloomy...g
dont be gloomy.
have the sweetest children. you are obviously a great mom. i loved reading your blog w/ their msgs to you. kudos.
ave the sweetest children. you are obviously a great mom. i loved reading your blog w/ their msgs to you. kudo
sorry!
sorry
finished crying
d crying
i seriously don`t know how
i seriously don`t know ho
they frighten me to death.........
they frighten me to death...
bad.....
bad.
#shortcakefail
#shortcakefai
- haha cute and beautiful u r cool miley.
- haha cute and beautiful u r cool miley
to piss them off?
to piss them off
what beautiful
t beautiful
what
hat
ww i wish
w i wish
damage done...about
damage done...a
can barely walk...
can barely walk.
mentioned - all these girls seem to be at a loss
d - all these girls seem to be at a los
**** break them
**** break th
and i`m alone
nd i`m alone
thanks okiebud! i get the message..sorry, the content yo

messenger, hate it.
r, hate it.
still i feel its not at all fair....
still i feel its not at all fair..
cute!!!
cute!!
awesome time.
awesome ti
cheap seats
cheap se
out enjoying the weather before
out enjoying the weather be
i`m feeling sick yuck!!!
i`m feeling sick yuck!
is better
s better
know i love robert pattinson!!
w i love robert pattinson!!
got an email from my auntie bout fathers day... mothers day w/seven moms but fathers day will consist of 2 dads #realitycheck
got an email from my auntie bout fathers day... mothers day w/seven moms but fathers day will consist of 2 dads #realitychec
good morning/bru
good morning/b
i wish i could go to e3. oh well....
i wish i could go to e3. oh well...
. hope all is well just thinking of you and wishing you the best! lots of love from over here in new york!
. hope all is well just thinking of you and wishing you the best! lots of love from over here in new york
... thanks
.. thanks
and humbug.
d humbug.
boo...i
boo.
! so sad
! so sa
i hate 

other fav song
er fav song
...are you ignoring me *frowns*
.are you ignoring me *frowns*
simply amazing. love it.
simply amazing. love it
. sorry.
. sorr
!! ur hurting my feelings
! ur hurting my feelings
we keep very different hours.
we keep very different hours
i hate when
i hate wh
no funny time...
no funny time..
miserable for
miserable fo
yay we`
yay we
enjoyed last night at what i call 'real world social networking' - lok event in vancouver. it was great!. but headeache today
enjoyed last night at what i call 'real world social networking' - lok event in vancouver. it was great!. but headeache toda
looks fabulous!
s fabulous
sick..l
sick..
get sum rest...g
get sum rest.
a crime?
a crime
desperate,that
desperate,t
good day...
good day..
i`m sunburnt arms are itching.
i`m sunburnt arms are itching
, sorry!
, sorry
looks good
s good
mother`s day present...a
mother`s day present..
were fab
e fab
... we make old look good!
.. we make old look good
gogol bordello < par. prolly not even

o, so all is still right in the world.
is sad that
is sad th
wtf!!!!
wtf!
**** i lost my star trek ring! starfleet academy class ring w/ golden gate bridge on it. got it as high school graduation present.
**** i lost my star trek ring! starfleet academy class ring w/ golden gate bridge on it. got it as high school graduation presen
just got back from tori`s. watching shark boy and lava girl
ust got back from tori`s. watching shark boy and lava girl
that i <3 ur music(
t i <3 ur music
good luck on finals everyone!!!!!!!!!
good luck on finals everyone!!
...but i haven`t unfollowed em` yet. i`m a loon.
.but i haven`t unfollowed em` yet. i`m a loon.
grrrr!!!
grrrr!!
sorry my friend ... but is paying the mortgage!
sorry my friend ... but is paying the mortgage
. watching everyone else act a fool is much better the not remembering acting a fool yourself
. watching everyone else act a fool is much better the not remembering acting a fool yoursel
but i can`t stop thinking about you
t i can`t s

be funny
e funny
have fun!
e fun
is making me hot chocolate with almond milk. mother day festivities is starting early!
is making me hot chocolate with almond milk. mother day festivities is starting earl
happy mothers
happy mo
there is always tomorrow.
there is always tomor
just harder
st harde
signs to be cool
s to be cool
hate worki
hate wo
you are just wrong!!!
you are just wrong!
awesome!!
awesome!
i miss jack n box and whataburger and oooo taco cabanaaaaaa lmao
i miss jack n box and whataburger and oooo taco cabanaaaaaa lm
dudes. thanks
es. thank
til i start crying. love this place so much.
l i start crying. love this place so much.
loves it that
loves it th
not impressed....!
not impressed..
...sorry abt nt gettin in touch..was hectic
.sorry abt nt gettin in touch..was hectic
you are pretty man...
you are pretty man..
sorry.
sorry
released stinks!
d stinks
can you say emotional? im really gonna miss mhs.
can you say emotional? im really gonna miss mh
`m so sad.
`m so sa
you enjo

'm down!
m down
hahaha!!
hahaha!
tgif but its so rainy and blah outside! i need sun trying to figure out what to do tonight, i want to go to prov to the hookah bar.
tgif but its so rainy and blah outside! i need sun trying to figure out what to do tonight, i want to go to prov to the hookah bar
an awesome
an awesom
being slammed with
being slammed wi
one day my hugs will come *fingers still crossed*
one day my hugs will come *fingers still crosse
like it`s not the most obvious thing though.
e it`s not the most obvious thing though
us glad
us gla
does anyone out there want to be really awesome and buy me one of these for my birthday? http:
does anyone out there want to be really awesome and buy me one of these for my birthday? ht
tired and
tired an
to repent!!
o repent!
re sore
e sore
e i love all the songs on
i love all the songs
burns...
burns..
#myweawkness chocolate...i
#myweawkness chocolate..
...but im on stupid vaca in the stupid cayman islands so i cnt go iloveyou<3
.but im on s

In [36]:
tweet = "......no internet"
selected_text = ".no internet"

# lower cased here
tweet = " ".join(str(tweet).split()).lower()
selected_text = " ".join(str(selected_text).split()).lower()
original_tweet = copy.deepcopy(tweet)
original_selected_text = copy.deepcopy(selected_text)

# some token optimization
special_tokens = ["!!!!", "!!!", "!!", "!", \
                  "....", "...", "..", ".", \
                  "????", "???", "??", "?", \
                  "****", "***", "**", "*", \
                  "----", "---", "--", "-", \
                  "~~~~", "~~~", "~~", "~", \
                  "////", "///", "//", "/", \
                  "````", "```", "``", "`", \
                  "====", "===", "==", "=", \
                  "####", "###", "##", "#", \
                  "&&&&", "&&&", "&&", "&", \
                  "))))", ")))", "))", ")", \
                  ";;;;", ";;;", ";;", ";", \
                  "''''", "'''", "''", "'", \
                  "<<<<", "<<<", "<<", "<"
                 ]
for token in special_tokens:
    replace_token = " ".join(token) + " "
    tweet = tweet.replace(token, replace_token)
    selected_text = selected_text.replace(token, replace_token)

tweet = " " + " ".join(tweet.split())
selected_text = " " + " ".join(selected_text.split())

len_st = len(selected_text) - 1
idx0 = None
idx1 = None

# get char idx
for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
    if " " + tweet[ind: ind + len_st] == selected_text:
        idx0 = ind
        idx1 = ind + len_st - 1
        break

# get char mask
char_targets = [0] * len(tweet)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1

# get word offsets
tweet_offsets_token_level = []
cursor = 0
input_ids_orig = []

for i, word in enumerate(tweet.split()):

    sub_words = tokenizer_1.tokenize(" " + word)
    encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
    number_of_tokens = len(encoded_word)
    input_ids_orig += encoded_word

    start_offsets = cursor
    end_offsets = cursor

    token_level_cursor = start_offsets

    for i in range(number_of_tokens):

        if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
            or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0:
                sub_word_len += 1
        else:
            sub_word_len = len(sub_words[i])

        tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
        cursor = token_level_cursor + sub_word_len
        token_level_cursor += sub_word_len

tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets
target_idx = []
for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
    if sum(char_targets[offset1: offset2]) > 0:
        target_idx.append(j)

targets_start = target_idx[0]
targets_end = target_idx[-1]


prediction = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]].strip()
filtered_output = ""
finished = False

# print("tweet: ", tweet)
# print("prediction: ", prediction)

if len(prediction) == 0:
    filtered_output = prediction
else:
    max_match = 0
    for idx in range(len(original_tweet)):
        curr_output = ""
        if original_tweet[idx] == prediction[0]:
            tweet_idx = idx
            for prediction_idx in range(len(prediction)):
                if (original_tweet[tweet_idx] == prediction[prediction_idx]):
                    curr_output += original_tweet[tweet_idx]
                    tweet_idx += 1

                    # reach tweet end
                    if (tweet_idx >= len(original_tweet)):
                        if (prediction_idx == len(prediction) - 1):
                            finished = True
                        break

                    # reach prediction end
                    if (prediction_idx == len(prediction) - 1):
                        finished = True
                        break
                else:
                    # skip extra " "
                    if (prediction[prediction_idx] == " "):
                        continue
                    else:
                        break
            if finished:
                filtered_output = curr_output
                break
            else:
                if max_match < len(curr_output):
                    max_match = len(curr_output)
                    filtered_output = curr_output

print(filtered_output, original_selected_text)

.no internet .no internet
